#BIG MAC INDEX
The Big Mac Index is an informal measure of currency exchange rates at ppp. It measures their value against a similar basket of goods and services, in this case a Big Mac. Differing prices at market exchange rates would imply that one currency is under or overvalued.

In [72]:
#https://www.investopedia.com/articles/fundamental-analysis/ppp-big-mac.asp

In [ ]:
# DUDA PARA ZAHID:
# La información que estoy leyendo es un excel de dos columnas, si quisiera agregar también a mi dataframe la columna de país, en donde la pongo en mi código


In [1]:
pip install quandl

In [2]:
#Importamos las librerías que vamos a usar
import pandas as pd
import numpy as np

import quandl

import time

from scipy import stats
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

In [3]:
#Ponemos el key access
quandl.ApiConfig.api_key = "NLgz3-aEXHxd5sSsdz-2"

In [4]:
#Cargamos google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#Cargamos el documento con los datos de los tickers y los países con los que vamos a trabajar
data_tickers = pd.read_excel('/content/drive/MyDrive/project_bigmac/bigmac_countries_full_list.xlsx')

In [6]:
#Vemos el dataframe para revisar como está compuesto
data_tickers.head()

,Ticker,Abreviatura,Pais
0,ECONOMIST/BIGMAC_ARG,ARG,Argentina
1,ECONOMIST/BIGMAC_AUS,AUS,Australia
2,ECONOMIST/BIGMAC_AUT,AUT,Austria
3,ECONOMIST/BIGMAC_BEL,BEL,Belgium
4,ECONOMIST/BIGMAC_BRA,BRA,Brazil


In [7]:
#Creamos una lista de tickers para realizar los request de información
tickers = list(data_tickers['Ticker'])

#Creamos una lista de países para incorporar a los datos
paises = list(data_tickers['Pais'])

#Fijamos los parámetros de las fechas en que haremos el request de información
start = '2018-01-01'
end = '2021-05-31'

In [8]:
#Creamos una función para obtener la informacion de quandl
def get_index(ticker):
    print('Obteniendo información de:', ticker)
    bigmac = quandl.get(ticker, start_date = start, end_date = end)
    bigmac['Ticker'] = ticker
    return bigmac

#Creamos una variable donde vamos a guardar la información que obtengamos de quandl
data_quandl = []

#Le pasamos la función a cada uno de nuestros tickers
for ticker in tickers:
    bigmac = get_index(ticker)
    data_quandl.append(bigmac)

Obteniendo información de: ECONOMIST/BIGMAC_ARG
Obteniendo información de: ECONOMIST/BIGMAC_AUS
Obteniendo información de: ECONOMIST/BIGMAC_AUT
Obteniendo información de: ECONOMIST/BIGMAC_BEL
Obteniendo información de: ECONOMIST/BIGMAC_BRA
Obteniendo información de: ECONOMIST/BIGMAC_GBR
Obteniendo información de: ECONOMIST/BIGMAC_CAN
Obteniendo información de: ECONOMIST/BIGMAC_CHL
Obteniendo información de: ECONOMIST/BIGMAC_CHN
Obteniendo información de: ECONOMIST/BIGMAC_COL
Obteniendo información de: ECONOMIST/BIGMAC_CRI
Obteniendo información de: ECONOMIST/BIGMAC_CZE
Obteniendo información de: ECONOMIST/BIGMAC_DNK
Obteniendo información de: ECONOMIST/BIGMAC_EGY
Obteniendo información de: ECONOMIST/BIGMAC_EST
Obteniendo información de: ECONOMIST/BIGMAC_EUR
Obteniendo información de: ECONOMIST/BIGMAC_FIN
Obteniendo información de: ECONOMIST/BIGMAC_FRA
Obteniendo información de: ECONOMIST/BIGMAC_DEU
Obteniendo información de: ECONOMIST/BIGMAC_GRC
Obteniendo información de: ECONOMIST/BIG

In [9]:
#Pasamos la información a un panda dataframe para iniciar el análisis
df = pd.concat(data_quandl, sort = True)
df.head()

,Ticker,dollar_adj_valuation,dollar_ex,dollar_ppp,dollar_price,dollar_valuation,euro_adj_valuation,local_price,sterling_adj_valuation,yen_adj_valuation,yuan_adj_valuation
Date,,,,,,,,,,,
2018-01-31,ECONOMIST/BIGMAC_ARG,18.6506,18.9375,14.204545,3.960396,-24.992499,5.87047,75.0,21.6375,54.6741,17.8301
2018-07-31,ECONOMIST/BIGMAC_ARG,-22.2,27.7250,13.611615,2.705140,-50.900000,-26,75.0,-15.1,1.2,-19.2
2019-01-31,ECONOMIST/BIGMAC_ARG,-41.9,37.4550,13.440860,2.002403,-64.100000,-43.4,75.0,-33.8,-26.1,-39.6
2019-07-31,ECONOMIST/BIGMAC_ARG,-22.9,41.8045,20.905923,2.870504,-50.000000,-20.1,120.0,-8.8,2.8,-12.5
2020-01-31,ECONOMIST/BIGMAC_ARG,-18.6,60.0656,30.158730,2.846887,-49.790000,-16.3,171.0,-11.2,7.3,-11.1


In [10]:
# Seleccionamos las columnas en dólares
df_bigmac = df[['Ticker', 'local_price', 'dollar_ex', 'dollar_price', 'dollar_ppp', 'dollar_valuation','dollar_adj_valuation']].reset_index()

In [11]:
df_bigmac

,Date,Ticker,local_price,dollar_ex,dollar_price,dollar_ppp,dollar_valuation,dollar_adj_valuation
0,2018-01-31,ECONOMIST/BIGMAC_ARG,75.0,18.9375,3.960396,14.204545,-24.992499,18.6506
1,2018-07-31,ECONOMIST/BIGMAC_ARG,75.0,27.7250,2.705140,13.611615,-50.900000,-22.2
2,2019-01-31,ECONOMIST/BIGMAC_ARG,75.0,37.4550,2.002403,13.440860,-64.100000,-41.9
3,2019-07-31,ECONOMIST/BIGMAC_ARG,120.0,41.8045,2.870504,20.905923,-50.000000,-22.9
4,2020-01-31,ECONOMIST/BIGMAC_ARG,171.0,60.0656,2.846887,30.158730,-49.790000,-18.6
...,...,...,...,...,...,...,...,...
304,2019-01-31,ECONOMIST/BIGMAC_VNM,65000.0,23199.0000,2.801845,11648.745520,-49.800000,NaN
305,2019-07-31,ECONOMIST/BIGMAC_VNM,65000.0,23231.0000,2.797985,11324.041812,-51.300000,NaN
306,2020-01-31,ECONOMIST/BIGMAC_VNM,66000.0,23176.0000,2.847774,11640.211640,-49.775000,NaN
307,2020-07-31,ECONOMIST/BIGMAC_VNM,66000.0,23180.0000,2.847282,11558.669002,-50.135000,NaN


In [12]:
#Vemos la información de las columnas a ver el tipo de datos de cada una, nombre de las columnas.
df_bigmac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  309 non-null    datetime64[ns]
 1   Ticker                309 non-null    object        
 2   local_price           309 non-null    float64       
 3   dollar_ex             309 non-null    float64       
 4   dollar_price          309 non-null    float64       
 5   dollar_ppp            309 non-null    float64       
 6   dollar_valuation      309 non-null    float64       
 7   dollar_adj_valuation  276 non-null    object        
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 19.4+ KB


In [13]:
#Vemos la información de las columnas a ver si existen datos nulos.
df_bigmac.isnull().sum()

Date                     0
Ticker                   0
local_price              0
dollar_ex                0
dollar_price             0
dollar_ppp               0
dollar_valuation         0
dollar_adj_valuation    33
dtype: int64

In [14]:
#Revisamos los datos nulos, para saber si es de un solo país o que sucede, a ver si se eliminan esos datos
df_bigmac[df_bigmac['dollar_adj_valuation'].isnull()]

,Date,Ticker,local_price,dollar_ex,dollar_price,dollar_ppp,dollar_valuation,dollar_adj_valuation
58,2018-01-31,ECONOMIST/BIGMAC_CRI,2290.0,568.530000,4.027932,433.712121,-23.713415,NaN
59,2018-07-31,ECONOMIST/BIGMAC_CRI,2290.0,567.800000,4.033110,415.607985,-26.800000,NaN
60,2019-01-31,ECONOMIST/BIGMAC_CRI,2290.0,606.975000,3.772808,410.394265,-32.400000,NaN
61,2019-07-31,ECONOMIST/BIGMAC_CRI,2290.0,581.935000,3.935147,398.954704,-31.400000,NaN
62,2020-01-31,ECONOMIST/BIGMAC_CRI,2350.0,569.965000,4.123060,414.462081,-27.283000,NaN
63,2020-07-31,ECONOMIST/BIGMAC_CRI,2350.0,581.825000,4.039015,411.558669,-29.264000,NaN
142,2018-01-31,ECONOMIST/BIGMAC_LVA,2.7,0.816827,3.305475,0.511364,-37.396307,None
143,2018-01-31,ECONOMIST/BIGMAC_LTU,2.8,0.816827,3.427900,0.530303,-35.077652,None
238,2018-01-31,ECONOMIST/BIGMAC_LKA,580.0,153.800000,3.771131,109.848485,-28.577058,NaN
239,2018-07-31,ECONOMIST/BIGMAC_LKA,580.0,159.270000,3.641615,105.263158,-33.900000,NaN


In [15]:
#Al observar que la columna 'dollar_adj_valuation' tiene 33 datos nulos, podemos dejarla.
#Si quisieramos eliminarla, cambiamos el 100 por un número menor a 33 en la función

def tratamiento_nulos(df):
  df_null = df_bigmac.copy()
  null_cols = df_null.isnull().sum()
  columns_to_drop = list(null_cols[null_cols > 30].index)
  df_null = df_null.drop(columns_to_drop, axis = 1)

  return df_null

In [16]:
#Guardamos el dataframe en uno nuevo que se llame df_bigmac_final

df_bigmac_final = tratamiento_nulos(df_bigmac)

In [17]:
#Revisamos la información
df_bigmac_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              309 non-null    datetime64[ns]
 1   Ticker            309 non-null    object        
 2   local_price       309 non-null    float64       
 3   dollar_ex         309 non-null    float64       
 4   dollar_price      309 non-null    float64       
 5   dollar_ppp        309 non-null    float64       
 6   dollar_valuation  309 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 17.0+ KB


In [18]:
#La columna de dollar_price es equivalente al precio de la hamburguesa en moneda extranjera, entre el tipo de cambio
#La columna de dollar_ppp es equivalente al tipo de cambio implícito
#La columna de dollar_valuation es equivalente al % de Sub/Sobre valuación de la moneda extranjera respecto al USD∫

In [19]:
#Vemos la información de las columnas a ver si existen datos nulos. 
#Si se decide eliminar la columna de dollar_adj_valuation, aquí debería aparecer en 0

df_bigmac_final.isnull().sum()

Date                0
Ticker              0
local_price         0
dollar_ex           0
dollar_price        0
dollar_ppp          0
dollar_valuation    0
dtype: int64

In [20]:
#Revisamos el nombre de las columnas que no tengan algún espacio que no hayamos detecatado
df_bigmac_final.columns = df_bigmac_final.columns.str.strip()

In [21]:
#Renombramos la columna de 'dollar_ex' por 'dollar_exchange'
df_bigmac_final = df_bigmac_final.rename(columns = {'dollar_ex':'dollar_exchange'})

In [22]:
#Revisamos el cambio
df_bigmac_final.head()

,Date,Ticker,local_price,dollar_exchange,dollar_price,dollar_ppp,dollar_valuation
0,2018-01-31,ECONOMIST/BIGMAC_ARG,75.0,18.9375,3.960396,14.204545,-24.992499
1,2018-07-31,ECONOMIST/BIGMAC_ARG,75.0,27.7250,2.705140,13.611615,-50.900000
2,2019-01-31,ECONOMIST/BIGMAC_ARG,75.0,37.4550,2.002403,13.440860,-64.100000
3,2019-07-31,ECONOMIST/BIGMAC_ARG,120.0,41.8045,2.870504,20.905923,-50.000000
4,2020-01-31,ECONOMIST/BIGMAC_ARG,171.0,60.0656,2.846887,30.158730,-49.790000


In [23]:
df_bigmac_final.to_excel('/content/drive/MyDrive/project_bigmac/df_bigmac_final.xlsx', index = False)

##Análisis 2021 - I Semestre


In [ ]:
#Hacemos un filtro para obtener la información del 2021 - I Semestre
bigmac_2021_1 = df_bigmac_final[df_bigmac_final['Date'] == '2021-01-31'].reset_index()

#Ordenamos los datos de acuerdo a la Sub/Sobre Valuación
bigmac_2021_1.sort_values('dollar_valuation', ascending = True).head(10)

,index,Date,Ticker,local_price,dollar_exchange,dollar_price,dollar_ppp,dollar_valuation
27,615,2021-01-31,ECONOMIST/BIGMAC_RUS,135.00,74.63000,1.808924,23.851590,-68.0
37,835,2021-01-31,ECONOMIST/BIGMAC_TUR,14.99,7.47050,2.006559,2.648410,-64.5
30,681,2021-01-31,ECONOMIST/BIGMAC_ZAF,33.50,15.52250,2.158158,5.918728,-61.9
38,857,2021-01-31,ECONOMIST/BIGMAC_UKR,62.00,28.14000,2.203269,10.954064,-61.1
16,373,2021-01-31,ECONOMIST/BIGMAC_IDN,34000.00,14125.00000,2.407080,6007.067138,-57.5
19,439,2021-01-31,ECONOMIST/BIGMAC_MYS,9.99,4.05200,2.465449,1.765018,-56.4
35,791,2021-01-31,ECONOMIST/BIGMAC_ROC,72.00,27.98000,2.573267,12.720848,-54.5
15,351,2021-01-31,ECONOMIST/BIGMAC_IND,190.00,73.39000,2.588909,33.568905,-54.3
13,307,2021-01-31,ECONOMIST/BIGMAC_HKG,20.50,7.75535,2.643337,3.621908,-53.3
20,461,2021-01-31,ECONOMIST/BIGMAC_MEX,54.00,20.11475,2.684597,9.540636,-52.6


##Análisis 2020 - II Semestre

In [ ]:
#Hacemos un filtro para obtener la información del 2020 - II Semestre
bigmac_2020_2 = df_bigmac_final[df_bigmac_final['Date'] == '2020-07-31'].reset_index()

#Ordenamos los datos de acuerdo a la Sub/Sobre Valuación
bigmac_2020_2.sort_values('dollar_valuation', ascending = True).head()

,index,Date,Ticker,local_price,dollar_exchange,dollar_price,dollar_ppp,dollar_valuation
30,680,2020-07-31,ECONOMIST/BIGMAC_ZAF,31.00,16.6725,1.859349,5.429072,-67.437
27,614,2020-07-31,ECONOMIST/BIGMAC_RUS,135.00,70.5850,1.912588,23.642732,-66.505
37,834,2020-07-31,ECONOMIST/BIGMAC_TUR,13.99,6.8595,2.039507,2.450088,-64.282
38,856,2020-07-31,ECONOMIST/BIGMAC_UKR,59.00,27.1300,2.174714,10.332750,-61.914
20,460,2020-07-31,ECONOMIST/BIGMAC_MEX,50.00,22.4360,2.228561,8.756567,-60.971


##Análisis México

In [28]:
#Fijamos las fechas en que haremos el request de información desde el 2000 que es la fecha más antigua con información de Méxcio
start = '2000-01-01'
end = '2021-05-31'

In [30]:
#Creamos una lista donde vamos a guardar la información que obtengamos de quandl
data_quandl_2 = []

#Le pasamos la función a cada uno de nuestros tickers
#En este caso solo sería el de México

tickers_2 = ['ECONOMIST/BIGMAC_MEX']

for ticker in tickers_2:
    bigmac = get_index(ticker)
    data_quandl_2.append(bigmac)

Obteniendo información de: ECONOMIST/BIGMAC_MEX


In [33]:
#Pasamos la información a un panda dataframe para iniciar el análisis
df_2 = pd.concat(data_quandl_2, sort = True)
df_2.head()

,Ticker,dollar_adj_valuation,dollar_ex,dollar_ppp,dollar_price,dollar_valuation,euro_adj_valuation,local_price,sterling_adj_valuation,yen_adj_valuation,yuan_adj_valuation
Date,,,,,,,,,,,
2000-04-30,ECONOMIST/BIGMAC_MEX,NaN,9.41,8.326693,2.22,-11.512293,NaN,20.9,NaN,NaN,NaN
2001-04-30,ECONOMIST/BIGMAC_MEX,NaN,9.29,8.622047,2.36,-7.190019,NaN,21.9,NaN,NaN,NaN
2002-04-30,ECONOMIST/BIGMAC_MEX,NaN,9.28,8.795181,2.37,-5.224346,NaN,21.9,NaN,NaN,NaN
2003-04-30,ECONOMIST/BIGMAC_MEX,NaN,10.53,8.487085,2.14,-19.400903,NaN,23.0,NaN,NaN,NaN
2004-05-31,ECONOMIST/BIGMAC_MEX,NaN,11.60,8.275862,2.08,-28.656361,NaN,24.0,NaN,NaN,NaN


In [35]:
#Seleccionamos solamente las columnas con las que vamos a trabajar
bigmac_mexico = df_2[['local_price', 'dollar_ex', 'dollar_price', 'dollar_ppp', 'dollar_valuation']].reset_index()
bigmac_mexico

,Date,local_price,dollar_ex,dollar_price,dollar_ppp,dollar_valuation
0,2000-04-30,20.9,9.41000,2.220000,8.326693,-11.512293
1,2001-04-30,21.9,9.29000,2.360000,8.622047,-7.190019
2,2002-04-30,21.9,9.28000,2.370000,8.795181,-5.224346
3,2003-04-30,23.0,10.53000,2.140000,8.487085,-19.400903
4,2004-05-31,24.0,11.60000,2.080000,8.275862,-28.656361
5,2005-06-30,28.0,10.86750,2.576490,9.150327,-15.800996
6,2006-01-31,28.0,10.54250,2.655917,8.888889,-15.685190
7,2006-05-31,29.0,11.30310,2.565668,9.354839,-17.236522
8,2007-01-31,29.0,10.88940,2.663140,9.006211,-17.293779
9,2007-06-30,29.0,10.76755,2.693277,8.504399,-21.018256


In [36]:
#Renombramos la columna de 'dollar_ex' por 'dollar_exchange'
bigmac_mexico = bigmac_mexico.rename(columns = {'dollar_ex':'dollar_exchange'})

In [37]:
#Exportamos a excel la información
bigmac_mexico.to_excel('/content/drive/MyDrive/project_bigmac/df_bigmac_mexico_final.xlsx', index = False)